In [ ]:
# !pip install --quiet openai

In [ ]:
# !pip install --upgrade openai

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import openai
categories = ['rec.sport.baseball', 'rec.sport.hockey']
sports_dataset = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories)

In [ ]:
print(sports_dataset['data'][0])

From: dougb@comm.mot.com (Doug Bank)
Subject: Re: Info needed for Cleveland tickets
Reply-To: dougb@ecs.comm.mot.com
Organization: Motorola Land Mobile Products Sector
Distribution: usa
Nntp-Posting-Host: 145.1.146.35
Lines: 17

In article <1993Apr1.234031.4950@leland.Stanford.EDU>, bohnert@leland.Stanford.EDU (matthew bohnert) writes:

|> I'm going to be in Cleveland Thursday, April 15 to Sunday, April 18.
|> Does anybody know if the Tribe will be in town on those dates, and
|> if so, who're they playing and if tickets are available?

The tribe will be in town from April 16 to the 19th.
There are ALWAYS tickets available! (Though they are playing Toronto,
and many Toronto fans make the trip to Cleveland as it is easier to
get tickets in Cleveland than in Toronto.  Either way, I seriously
doubt they will sell out until the end of the season.)

-- 
Doug Bank                       Private Systems Division
dougb@ecs.comm.mot.com          Motorola Communications Sector
dougb@nwu.edu       

In [ ]:
sports_dataset['target_names']

['rec.sport.baseball', 'rec.sport.hockey']

In [ ]:
sports_dataset.target_names[sports_dataset['target'][0]]

'rec.sport.baseball'

In [ ]:
len_all, len_baseball, len_hockey = len(sports_dataset.data), len([e for e in sports_dataset.target if e == 0]), len([e for e in sports_dataset.target if e == 1])
print(f"Total examples: {len_all}, Baseball examples: {len_baseball}, Hockey examples: {len_hockey}")

Total examples: 1197, Baseball examples: 597, Hockey examples: 600


DATA PREPARATION

In [ ]:
# sports_dataset['data']

In [ ]:
import pandas as pd

labels = [sports_dataset.target_names[x].split('.')[-1] for x in sports_dataset['target']]
texts = [text.strip() for text in sports_dataset['data']]
df = pd.DataFrame(zip(texts, labels), columns = ['prompt', 'completion'])
df.head()

,prompt,completion
0,From: dougb@comm.mot.com (Doug Bank)\nSubject:...,baseball
1,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,hockey
2,From: rudy@netcom.com (Rudy Wade)\nSubject: Re...,baseball
3,From: monack@helium.gas.uug.arizona.edu (david...,hockey
4,Subject: Let it be Known\nFrom: <ISSBTL@BYUVM....,baseball


In [ ]:
df['completion'][0]

'baseball'

In [ ]:
df.to_json("sports.jsonl", orient='records', lines=True)

In [ ]:
!openai tools fine_tunes.prepare_data -f sports.jsonl -q

Analyzing...

- Your file contains 1197 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 11 examples that are very long. These are rows: [134, 200, 281, 320, 404, 595, 704, 838, 1113, 1139, 1174]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts e

In [ ]:
ls

 sample_data/                       sports_prepared_train.jsonl
 sports.jsonl                      'sports_prepared_valid (1).jsonl'
'sports_prepared_train (1).jsonl'  'sports_prepared_valid (2).jsonl'
'sports_prepared_train (2).jsonl'  'sports_prepared_valid (3).jsonl'
'sports_prepared_train (3).jsonl'  'sports_prepared_valid (4).jsonl'
'sports_prepared_train (4).jsonl'   sports_prepared_valid.jsonl


In [ ]:
# pip install openai==0.28

In [ ]:
import openai
openai.api_key = ""

print(openai.Model.list())


{
  "object": "list",
  "data": [
    {
      "id": "gpt-4-0613",
      "object": "model",
      "created": 1686588896,
      "owned_by": "openai"
    },
    {
      "id": "gpt-4",
      "object": "model",
      "created": 1687882411,
      "owned_by": "openai"
    },
    {
      "id": "gpt-3.5-turbo",
      "object": "model",
      "created": 1677610602,
      "owned_by": "openai"
    },
    {
      "id": "gpt-4o-audio-preview-2025-06-03",
      "object": "model",
      "created": 1748908498,
      "owned_by": "system"
    },
    {
      "id": "gpt-4.1-nano",
      "object": "model",
      "created": 1744321707,
      "owned_by": "system"
    },
    {
      "id": "gpt-image-1",
      "object": "model",
      "created": 1745517030,
      "owned_by": "system"
    },
    {
      "id": "codex-mini-latest",
      "object": "model",
      "created": 1746673257,
      "owned_by": "system"
    },
    {
      "id": "gpt-4o-realtime-preview-2025-06-03",
      "object": "model",
      "created":

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [ ]:
!openai api files.create -f sports_prepared_train.jsonl -p fine-tune
!openai api files.create -f sports_prepared_valid.jsonl -p fine-tune

Upload progress: 100% 1.52M/1.52M [00:00<00:00, 522Mit/s]
{
  "object": "file",
  "id": "file-GGsiPVshmJ5GdiDuTAv8iu",
  "purpose": "fine-tune",
  "filename": "sports_prepared_train.jsonl",
  "bytes": 1519036,
  "created_at": 1749024308,
  "expires_at": null,
  "status": "processed",
  "status_details": null
}
Upload progress: 100% 387k/387k [00:00<00:00, 91.0Mit/s]
{
  "object": "file",
  "id": "file-QqGVjxfCZ2hPBXySicXKmF",
  "purpose": "fine-tune",
  "filename": "sports_prepared_valid.jsonl",
  "bytes": 387349,
  "created_at": 1749024310,
  "expires_at": null,
  "status": "processed",
  "status_details": null
}


In [ ]:
import openai

openai.api_key = ""

response = openai.FineTuningJob.create(
    training_file="file-GGsiPVshmJ5GdiDuTAv8iu",
    validation_file="file-QqGVjxfCZ2hPBXySicXKmF",
    model="gpt-3.5-turbo",
)

print(response)


{
  "object": "fine_tuning.job",
  "id": "ftjob-8VrlsYX0SrBA2TBRlJAeDqDA",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1749024920,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-6SPxjlH7Rp8KwVEVT1QnjqLO",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-QqGVjxfCZ2hPBXySicXKmF",
  "training_file": "file-GGsiPVshmJ5GdiDuTAv8iu",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": null,
  "seed": 282702743,
  "estimated_finish": null,
  "integrations": [],
  "metadata": null,
  "usage_metrics": null,
  "shared_with_openai": true,
  "eval_id": null,
  "method": {
    "type": "supervised",
    "supervised": {
      "hyperparameters": {
        "batch_size": "auto",
        "learning_rate_multiplier": "auto",
        "n_epochs": "auto"
      }
    }
  }
}


In [ ]:
import openai
print(openai.__version__)

0.28.0


In [ ]:
openai.FineTuningJob.list_events(id="ftjob-8VrlsYX0SrBA2TBRlJAeDqDA", stream=True)


<OpenAIObject list at 0x7c2a09272150> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-jPQTLWLCqHOfARTnJlLaZWwh",
      "created_at": 1749024953,
      "level": "error",
      "message": "The job failed due to an invalid training file. Invalid file format. Example 1 is missing key \"messages\".",
      "data": {
        "error_code": "invalid_training_file",
        "error_param": "training_file"
      },
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-hIyEmHy7S3I7ia1zlnnr3ouH",
      "created_at": 1749024920,
      "level": "info",
      "message": "Validating training file: file-GGsiPVshmJ5GdiDuTAv8iu and validation file: file-QqGVjxfCZ2hPBXySicXKmF",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-OTYcjpvrjaxWRHUNJ98H7yGw",
      "created_at": 1749024920,
      "level": "info",
      "message": "Created fin